# setup

In [1]:
!pip install transformers==4.38.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 23.7 MB/s eta 0:00:00


In [2]:
!pip install datasets evaluate accelerate eyecite --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.7/147.7 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.8/168.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/projects/us_supreme_court

/content/drive/MyDrive/projects/us_supreme_court


In [5]:
from utils.pre_train import *
from utils.notebook_utils import *

%load_ext autoreload
%autoreload 2

# Read from Hub

In [6]:
import os
from huggingface_hub import notebook_login
notebook_login()

In [16]:
dataset = load_dataset("raminass/opinions-94-23")
opinions = pd.DataFrame(dataset["train"])

# Filter Data

In [17]:
opinions=opinions[opinions.author=='per_curiam']

In [ ]:
# opinions = opinions[~opinions.author.isin(['Jackson', 'Barrett'])] # 15 model
# opinions = opinions[~opinions.author.isin(['Jackson', 'Barrett', 'Kavanaugh'])] # 14 model

In [ ]:
opinions.type.value_counts()

majority      2304
dissenting    1327
concurring     848
Name: type, dtype: int64

In [ ]:
opinions.author.value_counts()

Thomas       571
Scalia       473
Breyer       443
Stevens      407
Ginsburg     390
Kennedy      326
Alito        286
Souter       230
Sotomayor    226
Gorsuch      168
O'Connor     167
Kavanaugh    160
Kagan        145
Rehnquist    144
Roberts      123
Barrett      122
Jackson       98
Name: author, dtype: int64

# Add Labels

In [14]:
id2label, label2id = get_labels_maping(opinions)

In [15]:
opinions["label"] = opinions["author"].map(label2id)

In [ ]:
opinions.sample(3)

,case_name,text,year,author,type,clean_text,label
4153,Miller v. Alabama,I join the Court’s opinion in full. I add that...,2012,Breyer,concurring,I join the Court’s opinion in full. I add that...,5
2058,Puerto Rico v. Sanchez Valle,"I join in full the Court’s opinion, which coge...",2016,Ginsburg,concurring,"I join in full the Court’s opinion, which coge...",11
2895,Day v. McDonough,The Court today disregards the Federal Rules o...,2006,Scalia,dissenting,The Court today disregards the Federal Rules o...,12


In [ ]:
import json
with open("data/j_year.json", "r") as j:
    judges_by_year = json.loads(j.read())
labels_by_year = {k: [label2id[item] for item in v] for k,v in judges_by_year.items()}

# Train-Test

In [ ]:
from sklearn.model_selection import train_test_split
opinions_train, opinions_test = train_test_split(opinions, random_state=2024, test_size=0.2, stratify=opinions['label'])
# opinions_train, opinions_test = train_test_split(opinions, test_size=0.2, stratify=opinions['label'])

In [ ]:
opinions_test.author.value_counts()

Thomas       114
Scalia        95
Breyer        89
Stevens       81
Ginsburg      78
Kennedy       65
Alito         57
Souter        46
Sotomayor     45
Gorsuch       34
O'Connor      33
Kavanaugh     32
Rehnquist     29
Kagan         29
Roberts       25
Barrett       24
Jackson       20
Name: author, dtype: int64

In [ ]:
opinions_train.author.value_counts()

Thomas       457
Scalia       378
Breyer       354
Stevens      326
Ginsburg     312
Kennedy      261
Alito        229
Souter       184
Sotomayor    181
O'Connor     134
Gorsuch      134
Kavanaugh    128
Kagan        116
Rehnquist    115
Roberts       98
Barrett       98
Jackson       78
Name: author, dtype: int64

# Split Paragraphs

In [ ]:
par_train = split_data(opinions_train, id2label, label2id)
par_test = split_data(opinions_test, id2label, label2id)

In [ ]:
par_train.author.value_counts()

Thomas       3998
Breyer       3420
Scalia       3334
Kennedy      3072
Alito        2920
Ginsburg     2833
Stevens      2271
Sotomayor    2037
Souter       1750
Kagan        1542
O'Connor     1296
Jackson      1214
Gorsuch      1206
Roberts      1124
Kavanaugh    1100
Rehnquist     848
Barrett       745
Name: author, dtype: int64

In [ ]:
par_test.author.value_counts()

Breyer       960
Thomas       941
Kennedy      910
Ginsburg     756
Scalia       751
Alito        662
Stevens      569
Sotomayor    496
Souter       466
Kagan        371
O'Connor     365
Gorsuch      352
Roberts      327
Kavanaugh    275
Rehnquist    237
Jackson      220
Barrett      176
Name: author, dtype: int64

In [ ]:
# par_train = par_train.groupby('author',as_index = False,group_keys=False).apply(lambda s: s.sample(2000, replace=len(s)<2000))

In [ ]:
par_train.author.value_counts()

Thomas       3998
Breyer       3420
Scalia       3334
Kennedy      3072
Alito        2920
Ginsburg     2833
Stevens      2271
Sotomayor    2037
Souter       1750
Kagan        1542
O'Connor     1296
Jackson      1214
Gorsuch      1206
Roberts      1124
Kavanaugh    1100
Rehnquist     848
Barrett       745
Name: author, dtype: int64

In [ ]:
# par_test = par_test.groupby('author',as_index = False,group_keys=False).apply(lambda s: s.sample(400, replace=len(s)<400))

In [ ]:
par_test.author.value_counts()

Breyer       960
Thomas       941
Kennedy      910
Ginsburg     756
Scalia       751
Alito        662
Stevens      569
Sotomayor    496
Souter       466
Kagan        371
O'Connor     365
Gorsuch      352
Roberts      327
Kavanaugh    275
Rehnquist    237
Jackson      220
Barrett      176
Name: author, dtype: int64

# Model

In [ ]:
from utils import train

In [ ]:
from datasets.utils.logging import disable_progress_bar
disable_progress_bar()
ds_dict = {
        "train": Dataset.from_pandas(
            par_train[["label", "text"]], preserve_index=False
        ),
        "test": Dataset.from_pandas(
            par_test[["label", "text"]], preserve_index=False
        ),
    }
ds = DatasetDict(ds_dict)
tokenized_court = train.tokenize_dataset(ds)

# Analysis

In [ ]:
from transformers import AutoModelForSequenceClassification
from utils import train

trainers = {}
for i in [1,2,3,4,7,8]:
  trainers[f'trainer_{i}'] = train.load_trainer(
    id2label,
    label2id,
    freeze=False,
    epochs=5,
    model_name=f"raminass/m{i}",
    batch_size=16,
    push_to_hub=False)

In [ ]:
from scipy.special import softmax
pred = {k:[] for k,v in trainers.items()}
for index, row in opinions.iterrows():
  chunks = split_text(row["clean_text"])
  list_ds = [{"text":c} for c in chunks]
  tokenized_case = train.tokenize_dataset(Dataset.from_list(list_ds))
  for k,v in trainers.items():
    prop = softmax(softmax(v.predict(tokenized_case).predictions, axis=1).mean(axis=0))
    pred[k].append(prop)


In [64]:
import numpy as np

def symmetric_kl_divergence(p, q):
    kl_pq = np.sum(p * np.log(p / q))
    kl_qp = np.sum(q * np.log(q / p))
    return 0.5 * (kl_pq + kl_qp)

In [67]:
ditances = {}
for k,v in pred.items():
  ditances[k] = 0
  for m,n in pred.items():
    ditances[k] =ditances[k]+ sum([symmetric_kl_divergence(v[i],n[i]) for i in range(len(v))])

In [68]:
ditances

{'trainer_1': 1.3973875555691393,
 'trainer_2': 1.0959024279856067,
 'trainer_3': 1.322325456947624,
 'trainer_4': 1.095929196705356,
 'trainer_7': 1.264765505959076,
 'trainer_8': 1.1440099073051897}

In [ ]:
# select 4 due to best on Kavanaugh